In [29]:
# !pip install --upgrade jax[cuda] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [2]:
# using Kernel PyTorch-2.2.0

import os
import csv
import jax
import haiku as hk
import numpy as np
import pandas as pd

# import torch
# import pickle
# import swifter
# import seaborn as sns
# from tqdm import tqdm

import jax.numpy as jnp
import matplotlib.pyplot as plt

import multiprocessing as mp

from datetime import datetime, timedelta

from nucleotide_transformer.pretrained import get_pretrained_model

# device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device

from datasets import Dataset, DatasetDict

print(jax.devices())



# import os
# import jax
# # import torch
# # import pickle
# import pandas as pd
# import haiku as hk
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import jax.numpy as jnp
# import matplotlib.pyplot as plt

# from tqdm import tqdm
# from sklearn.preprocessing import StandardScaler
# from transformers import AutoModel #, AutoModelForMaskedLM
# from nucleotide_transformer.pretrained import get_pretrained_model

# device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device

/home/sunhuaikuan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[CudaDevice(id=0)]


In [3]:
try:
    import nucleotide_transformer
except:
    !pip install numpy==1.23.5
    !pip install git+https://github.com/instadeepai/nucleotide-transformer@main |tail -n 1
    import nucleotide_transformer

if "COLAB_TPU_ADDR" in os.environ:
    from jax.tools import colab_tpu

    colab_tpu.setup_tpu()

In [4]:
comp = {'A':1, 'C':2, 'G':3, 'T':4}

#@title Select a model
#@markdown ---
model_name = '50M_multi_species_v2'#@param['500M_human_ref', '500M_1000G', '2B5_1000G', '2B5_multi_species', '50M_multi_species_v2', '100M_multi_species_v2', '250M_multi_species_v2', '500M_multi_species_v2', '1B_agro_nt']
model_name = '500M_human_ref'#@param['500M_human_ref', '500M_1000G', '2B5_1000G', '2B5_multi_species', '50M_multi_species_v2', '100M_multi_species_v2', '250M_multi_species_v2', '500M_multi_species_v2', '1B_agro_nt']
#@markdown ---

# Get pretrained model
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    embeddings_layers_to_save=(20,),
    attention_maps_to_save=((1, 4), (7, 18)),
    max_positions=32,
    # If the progress bar gets stuck at the start of the model wieghts download,
    # you can set verbose=False to download without the progress bar.
    verbose=False
)
forward_fn = hk.transform(forward_fn)

In [5]:
# Get data and tokenize it
# sequences = [
#     "ATTCCGAAATCGCTGACCGATCGTACGAAA",
#     "ATTTCTCTCTCTCTCTGAGATCGATCGATCGATATCTCTCGAGCTAGC",
# ]
# tokens_ids = [b[1] for b in tokenizer.batch_tokenize(sequences)]
# tokens_str = [b[0] for b in tokenizer.batch_tokenize(sequences)]
# tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)

In [6]:
# !pip install jax jaxlib==0.1.87+cuda110 -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [7]:
# import jax
# import jax.numpy as jnp

# x = jnp.ones((1000, 1000))
# y = jnp.dot(x, x.T).block_until_ready()

# %%time
# # Initialize random key
# random_key = jax.random.PRNGKey(0)

# # Infer
# outs = forward_fn.apply(parameters, random_key, tokens)

In [19]:
def append_data(final_df, sub_df, sub_embedding_df):
    
    sub_df=sub_df.drop(columns=['sequence'])
    sub_df = sub_df.reset_index(drop=True)
    sub_embedding_df = sub_embedding_df.reset_index(drop=True)
    
    sub_final_df = pd.concat([sub_embedding_df, sub_df],  axis=1, ignore_index=True)
    final_df = pd.concat([final_df, sub_final_df],  axis=0, ignore_index=True) 
    
    return final_df

In [9]:
def get_tokens(df):
    sequences = []
    # refs=[]
    for index, row in df.iterrows():      
        # subsequence = get_subsequence(chrom_sequences, chrom, pos_start, length)
        subsequence = row['sequence']
        if 'N' in subsequence:
            print("The character 'N' is present in the string.")
    
        sequences.append(subsequence)
        # refs.append(ref)
    
    print (len(sequences))
    # sequences

    try:
        tokens_ids = [b[1] for b in tokenizer.batch_tokenize(sequences)]
        tokens_str = [b[0] for b in tokenizer.batch_tokenize(sequences)]
        tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)   

    except Exception as e:
        print(f"exception caught: {e}"+str(row['CHROM'])+'-'+str(row['START'])+'-'+str(row['SIZE']))
        # print(f"exception caught: {e}")
        tokens=None
        
    return tokens

In [10]:
def get_embeddings(tokens):

    # Initialize random key
    random_key = jax.random.PRNGKey(0)
    
    # Infer
    outs = forward_fn.apply(parameters, random_key, tokens)    
    # print(outs.keys())
    # print(outs["embeddings_20"].shape)
    # outs["embeddings_20"]
    
    # my_embedding=outs["embeddings_20"][:,16,:]
    my_embedding=outs["embeddings_20"][:,0,:]
    
    my_embedding.shape
    column_names = [f'{i}' for i in range(0, my_embedding.shape[1])]
    embedding_df = pd.DataFrame(my_embedding, columns=column_names)
    return embedding_df

### load dna sequence data file

In [24]:
import pandas as pd

# pathogenecity_type='noncoding'
pathogenecity_type='missense'

df=pd.read_csv('dna_segment_'+pathogenecity_type+'.csv')
# df

In [25]:
%%time

sub_df = pd.DataFrame()    
final_df = pd.DataFrame()
segment=2000


# csv_Filename = './homo_sapiens_nt_embedding.csv'
# if os.path.exists(csv_Filename):
#     os.remove(csv_Filename)

max_length= 186

now = datetime.now()
formatted_time = now.strftime("%y-%m-%d-%H-%M-%S")
csv_filename = './pathogenecity_nt_'+pathogenecity_type+'_'+formatted_time+'.csv'


cnt=0
for index, row in df.iterrows():
    cnt+=1
    sub_df = sub_df.drop(sub_df.index)
    
    if (cnt % segment==0):
        sub_df = df.iloc[cnt-segment:cnt]
        sub_tokens = get_tokens(sub_df)
        sub_embedding_df = get_embeddings(sub_tokens)       

        final_df = append_data(final_df, sub_df, sub_embedding_df)
        
        sub_df = sub_df.reset_index(drop=True)
        print(f"complete batch...... {cnt}")


print(f"last index...... {(cnt)}")
sub_df = df.iloc[cnt-(cnt % segment):cnt]
sub_tokens = get_tokens(sub_df)
sub_embedding_df = get_embeddings(sub_tokens)        
final_df = append_data(final_df, sub_df, sub_embedding_df)

final_df.to_csv(csv_filename, sep=',', index=False,  header=True, na_rep='NaN')

2000
Before attention blocks: [[[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [-0.6353631   1.5822513  -0.9913084  ... -0.61847615  0.379358
    0.0855906 ]
  [-1.0689963   1.8419149   2.3481312  ...  0.11870462  0.35518837
   -0.5532786 ]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]]

 [[-0.2133187  -0.37896746  0.9894879  ... -1.4485674   0.29604977
   -0.8132925 ]
  [-0.44678867  2.5200481  -2.0669289  ... -0.16582397 -0.42979893
   -0.84155095]
  [-1.2272452   1.3481911   0.32137865 ...  0.7991009   0.77097625
   -0.8474954 ]
  ...
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864607  -1.9107313
   -0.23773763]
  [ 1.7484777  -0.23501392  1.5967557  ...  1.4864

In [14]:
# csv_filename = './pathogenecity_nt_'+pathogenecity_type+'_'+formatted_time+'.csv'
# final_df.to_csv(csv_filename, sep=',', index=False,  header=True, na_rep='NaN')

### Load CSV File

In [ ]:
import pandas as pd
# csv_filename='./pathogenecity_nt_missense.csv'
def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    
    column_names = [f'{i}' for i in range(0, df.shape[1]-1)]
    column_names.extend(['y'])
    
    df.columns = column_names
    return df

df = load_embedding_file(csv_filename)
df.head(5)

In [23]:
final_df

,0,1,2,3,4,5,6,7,8,9,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,1.519376,14.057747,3.488417,10.252307,-13.983130,6.598228,-8.648248,9.700171,3.890464,36.742058,...,-0.167834,-9.328757,-1.683411,5.836273,-8.055669,-22.768402,-14.255890,-5.055781,-2.236104,0
1,1.942038,15.880097,1.786344,14.161925,-10.093940,3.963628,-5.675667,9.595583,4.319385,40.395947,...,5.784422,-7.271989,0.122954,6.074058,-3.701434,-20.236513,-16.867464,-0.694164,3.766532,0
2,0.235975,15.086056,1.359051,11.957237,-9.942860,4.851335,-7.758008,10.239451,2.692015,41.241138,...,2.834705,-9.811299,-3.168976,5.549085,-6.191209,-18.261349,-10.915918,-5.920380,2.439998,0
3,-4.232018,19.112915,3.011271,11.852527,-8.105716,4.616992,-9.162451,8.860380,6.024180,39.067871,...,-0.107898,-4.775982,1.403075,4.807008,-2.878212,-18.467976,-12.042815,-7.322570,4.602901,0
4,-4.232018,19.112915,3.011271,11.852527,-8.105716,4.616992,-9.162451,8.860380,6.024180,39.067871,...,-0.107898,-4.775982,1.403075,4.807008,-2.878212,-18.467976,-12.042815,-7.322570,4.602901,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95755,1.219426,10.794795,6.606981,1.585570,-10.872110,4.278807,-7.358446,6.307883,4.438954,37.420662,...,0.823933,-3.972571,1.463499,3.080049,-6.443473,-23.649530,-10.920424,-4.820085,2.839149,0
95756,-2.612770,11.947458,2.890443,7.339174,-10.585886,3.848897,-2.906900,10.856840,0.500175,36.630249,...,-0.417426,-3.469111,-1.041096,4.790893,-6.180765,-22.434631,-10.472472,-6.724950,2.036397,0
95757,-1.100140,13.092549,6.563645,7.606856,-8.716226,-1.079019,-7.184974,10.401398,1.287622,41.700886,...,-0.661174,-2.583191,0.748601,6.817532,-4.297803,-21.977655,-12.254357,-4.299364,1.909014,0
95758,0.800380,13.502211,-3.412460,6.424632,-9.330681,4.252857,-8.148738,12.095390,0.680981,43.453564,...,0.967206,-2.596486,2.392622,3.782078,-4.181155,-22.223484,-11.513384,-4.036939,-1.742803,0
